# 03.04 Basic Models in TensorFlow (Logistic Regression + placehold + CE) 

### import 구문

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

### 모델의 파라메터들을 선언

In [4]:
learning_rate = 0.01
batch_size = 128
n_epochs = 30

### Step 1: 데이터를 읽는다.

In [5]:
# TF Learn's에 있는 함수로 MNIST 데이터를 data/mnist에 가져온다.
mnist = input_data.read_data_sets('data/minst', one_hot=True) # one hot으로 정답이 표현된 값이다.
X_batch, Y_batch = mnist.train.next_batch(batch_size) # 배치 사이즈 만큼 가져온다.

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/minst/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/minst/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/minst/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/minst/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

### Step 2: placeholders를 만든다.

In [6]:
# 각 이미지 MNIST 데이터의 형태는 28*28 = 784 이다.
# 그러므로, 각 이미지의 표현은 1*784 tensor로 표현한다.
# 각이미지의 10개의 종류 중 하나이며 , 숫자 0-9 사이 하나와 일치한다.
# 각 라벨은 one hot vector 이다.
X = tf.placeholder(tf.float32, [batch_size, 784], name='image')
Y = tf.placeholder(tf.int32, [batch_size, 10], name='label')

### Step 3: weights 와 bias를 만든다.

In [7]:
# 평균이 0이고 표준편차가 0.01인 랜덤한 w로 초기화 한다.
# b는 초기화 0으로 한다.
# w는 X와 Y에 의존하므로 Y = tf.matmul(X, w) 한다.
# b는 Y에 의존적이다.
w = tf.get_variable(name='weights', shape=(784, 10), initializer=tf.random_normal_initializer(0, 0.01))
b = tf.get_variable(name='bias', shape=(1, 10), initializer=tf.zeros_initializer())

### Step 4: 모델 구성

In [8]:
# 모델은 log를 리턴한다.
# 해당 log는 나중에 softmax layer를 통과한다.
logits = tf.matmul(X, w) + b

### Step 5: 손실 함수 정의

In [9]:
# 손실 함수로서 softmax_cross_entropy_with_logits를 사용한다.
entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y, name='loss')
loss = tf.reduce_mean(entropy) # 배치의 모든 값에 대한 평균을 계산한다.

### Step 6: 훈련 옵티마이져 선언

In [10]:
# 아담 옵티마이져를 사용한다. loss가 최소가 될때 까지 0.01씩 gradient descent를 진행 한다 .
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

### Step 7: 테스트 세트를 통해서 정확도를 계산한다.

In [11]:
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

In [12]:
writer = tf.summary.FileWriter('./graphs/logreg_placeholder', tf.get_default_graph())
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples/batch_size) # 전체 데이터를 몇번의 iterator를 해야 하는지 계산한다.
    
    # 훈련 모델 몇번 epochs 할지
    for i in range(n_epochs):
        total_loss = 0
        
        for j in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size) # 배치 크기 만큼 다음 데이터를 가져온다.
            _, loss_batch = sess.run([optimizer, loss], {X: X_batch, Y: Y_batch}) # 배치 전체의 손실을 가져온다. Placehold구문에서는 해당 부분 처럼 값을 넣어준다.
            total_loss += loss_batch 
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    print('Total time: {0} seconds'.format(time.time() - start_time))
    
    # 테스트 모델
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run(accuracy, {X: X_batch, Y: Y_batch})
        total_correct_preds += accuracy_batch
        
    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))
writer.close()

Average loss epoch 0: 0.367365970408722
Average loss epoch 1: 0.29471331838266557
Average loss epoch 2: 0.2885190508304498
Average loss epoch 3: 0.27809937867946
Average loss epoch 4: 0.27509605710878793
Average loss epoch 5: 0.27040455556346543
Average loss epoch 6: 0.27115142989756064
Average loss epoch 7: 0.2689427288296895
Average loss epoch 8: 0.26644441096376986
Average loss epoch 9: 0.26742811810164463
Average loss epoch 10: 0.26422464467220375
Average loss epoch 11: 0.26262947395964936
Average loss epoch 12: 0.2626580724488327
Average loss epoch 13: 0.2569377312397623
Average loss epoch 14: 0.2593336058107567
Average loss epoch 15: 0.2582681898401056
Average loss epoch 16: 0.26080711442968507
Average loss epoch 17: 0.25938706289290825
Average loss epoch 18: 0.25599642653704247
Average loss epoch 19: 0.2542679260695453
Average loss epoch 20: 0.2589911801310686
Average loss epoch 21: 0.25336476512652734
Average loss epoch 22: 0.2584623379029316
Average loss epoch 23: 0.2523753197